In [1]:
%pip install python-dotenv pandas numpy


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain langchain-community langchain-core langchain-openai  openai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip list | grep langchain

langchain                 0.2.16
langchain-community       0.2.16
langchain-core            0.2.38
langchain-openai          0.1.23
langchain-text-splitters  0.2.4

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
def save_string_to_file(content, filename):
    with open(filename, 'w') as file:
        file.write(content)

In [2]:
import dotenv
import os
dotenv.load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

from langchain.chains import RefineDocumentsChain, MapReduceDocumentsChain, LLMChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.schema.runnable import RunnableSequence
from langchain.schema import StrOutputParser

In [3]:
# Initialize the LLM
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50000, chunk_overlap=200)

# Split your text into chunks
with open('../raw/transcripts-podcast/andrew_callaghan_transcript.txt', 'r') as file:
    text = file.read()
docs = text_splitter.create_documents([text])

print(len(docs))

6


In [9]:
# User command
user_command="""
Analysis the podcast episode from the Lex Fridman Podcast series. Summarise the key points discussed in the episode with a focus on: 
1) The main arguments presented 
2) Any notable quotes from the guest 
3) Relevant topics or themes covered
Ensure the summary is clear and concise, only response the above 3 sections.
"""

## RefineDocumentsChain

In [11]:
# Define prompts
document_prompt = PromptTemplate(input_variables=["page_content"], template="{page_content}")
initial_prompt = PromptTemplate.from_template(user_command + ": {context}")
initial_chain = LLMChain(llm=llm, prompt=initial_prompt)

refine_prompt = PromptTemplate.from_template(
    "Here's a summary so far: {prev_response}\n"
    "Now refine it with this additional context: {context}\n"
    user_command
)
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

# Create the RefineDocumentsChain
chain = RefineDocumentsChain(
    initial_llm_chain=initial_chain,
    refine_llm_chain=refine_chain,
    document_prompt=document_prompt,
    document_variable_name="context",
    initial_response_name="prev_response",
    verbose=True
)

# Generate the summary
summary_refine = chain.invoke(docs)
print(len(summary_refine))

/var/folders/y6/g14l2wls2tb0fyw1k42qsspr0000gq/T/ipykernel_77335/4186316529.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  initial_chain = LLMChain(llm=llm, prompt=initial_prompt)




> Entering new RefineDocumentsChain chain...

> Finished chain.
2


## MapReduceDocumentsChain

In [10]:
map_prompt = PromptTemplate.from_template(user_command + "{context}")
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_prompt = PromptTemplate.from_template(user_command + "{context}")
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="context"
)

reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=4000
)

chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="context"
)

# Generate the summary
summary_mapreduce = chain.invoke(docs)
print(len(summary_mapreduce))

2


In [11]:
from IPython.display import display_markdown
display_markdown(summary_mapreduce['output_text'], raw=True)

### Key Points Discussed

1. **Main Arguments Presented:**
   - **Journalism as a Therapeutic Mechanism:** Andrew Callaghan discusses how journalism has been a therapeutic outlet for him, especially in dealing with his Hallucinogen Persisting Perception Disorder (HPPD) and depersonalization.
   - **Exploration of Extremes:** Callaghan emphasizes his interest in exploring the fringes of society, from QAnon adherents to residents of O-Block, and how these experiences validate his sense of reality.
   - **Critique of Traditional Education:** Both Callaghan and Fridman critique the traditional education system, highlighting its rigidity and lack of engagement for students with specific interests.
   - **Substance Use and Creativity:** The conversation touches on the role of substances like alcohol and drugs in creative processes, with Callaghan reflecting on his past substance use and its impact on his work.

2. **Notable Quotes from the Guest:**
   - "Being a journalist gives you a ticket to everywhere that you want to go in life. It allows you to step into different realities almost and then go back to yours and it just keeps life interesting."
   - "I think fame getting to your head. If you spend more than a hundred bucks on sunglasses, you've officially gone off the deep end."
   - "The open road never goes anywhere, and it's kind of like, I see an invisible door in the corner of the room all the time. That makes me more comfortable because I'm like, 'Hey, at the end of the day, if I'm bummed out, I can go hit the road and I'm sure there's going to be a fun time ahead.'"

3. **Relevant Topics or Themes Covered:**
   - **HPPD and Mental Health:** Callaghan's experience with HPPD and its psychological effects, including depersonalization and derealization.
   - **Gonzo Journalism:** The influence of gonzo journalism and figures like Hunter S. Thompson on Callaghan's work.
   - **Substance Use:** The discussion on the role of alcohol and drugs in creative processes and personal life.
   - **Education System:** Critique of the traditional education system and the need for more engaging and interest-based learning.
   - **Hitchhiking and Vagabonding:** Callaghan's hitchhiking journey across the United States and the insights gained from that experience.
   - **Existential Reflections:** Reflections on the insignificance of human existence in the grand scheme of the universe, touching on topics like space and black holes.

The episode provides a deep dive into Andrew Callaghan's unique perspective on journalism, mental health, and the human experience, enriched by his personal anecdotes and philosophical reflections.

In [12]:
save_string_to_file(summary_mapreduce['output_text'], "gpt4o-mapreduce.md")

In [26]:
from langchain.chains import MapReduceDocumentsChain

# Initialize the MapReduce chain with example parameters
map_reduce_chain = MapReduceDocumentsChain(max_docs=100, reducer_config={'method': 'avg'})

# Invoke the chain to process the documents
summary_mapreduce = map_reduce_chain.invoke(docs)
print(summary_mapreduce['output_text'])

ValidationError: 1 validation error for MapReduceDocumentsChain
__root__
  llm_chain must be provided (type=value_error)